# 1 - Data Access and Download

In [1]:
import os, sys, re, ast
import pandas as pd
import numpy as np
import xarray as xr

from ooi_data_explorations.common import load_kdata

from ooinet import M2M

### Available Gliders and Datasets

##### ADCP Datasets

In [2]:
adcp_datasets = M2M.search_datasets(array='CP05MOAS', instrument='ADCP', English_names=True)
adcp_datasets

Searching https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP05MOAS


,array,array_name,node,node_name,instrument,instrument_name,refdes,url,deployments
0,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL559,Coastal Glider 559,01-ADCPAM000,Velocity Profiler (600kHz),CP05MOAS-GL559-01-ADCPAM000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5]"
1,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL514,Coastal Glider 514,01-ADCPAM000,Velocity Profiler (600kHz),CP05MOAS-GL514-01-ADCPAM000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3]"
2,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL389,Coastal Glider 389,01-ADCPAM000,Velocity Profiler (600kHz),CP05MOAS-GL389-01-ADCPAM000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6]"
3,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL388,Coastal Glider 388,01-ADCPAM000,Velocity Profiler (600kHz),CP05MOAS-GL388-01-ADCPAM000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]"
4,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL387,Coastal Glider 387,01-ADCPAM000,Velocity Profiler (600kHz),CP05MOAS-GL387-01-ADCPAM000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8]"
5,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL380,Coastal Glider 380,01-ADCPAM000,Velocity Profiler (600kHz),CP05MOAS-GL380-01-ADCPAM000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]"
6,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL379,Coastal Glider 379,01-ADCPAM000,Velocity Profiler (600kHz),CP05MOAS-GL379-01-ADCPAM000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
7,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL376,Coastal Glider 376,01-ADCPAM000,Velocity Profiler (600kHz),CP05MOAS-GL376-01-ADCPAM000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
8,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL374,Coastal Glider 374,01-ADCPAM000,Velocity Profiler (600kHz),CP05MOAS-GL374-01-ADCPAM000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5]"
9,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL340,Coastal Glider 340,01-ADCPAM000,Velocity Profiler (600kHz),CP05MOAS-GL340-01-ADCPAM000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


##### Engineering Datasets

In [3]:
eng_datasets = M2M.search_datasets(array='CP05MOAS', instrument='ENG', English_names=True)
eng_datasets

Searching https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP05MOAS


,array,array_name,node,node_name,instrument,instrument_name,refdes,url,deployments
0,CP05MOAS,Coastal Pioneer NES Mobile Assets,PG583,Profiling Glider 583,00-ENG000000,Mobile Asset Controller,CP05MOAS-PG583-00-ENG000000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4]"
1,CP05MOAS,Coastal Pioneer NES Mobile Assets,PG564,Profiling Glider 564,00-ENG000000,Mobile Asset Controller,CP05MOAS-PG564-00-ENG000000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5]"
2,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL559,Coastal Glider 559,00-ENG000000,Mobile Asset Controller,CP05MOAS-GL559-00-ENG000000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5]"
3,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL514,Coastal Glider 514,00-ENG000000,Mobile Asset Controller,CP05MOAS-GL514-00-ENG000000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3]"
4,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL389,Coastal Glider 389,00-ENG000000,Mobile Asset Controller,CP05MOAS-GL389-00-ENG000000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6]"
5,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL388,Coastal Glider 388,00-ENG000000,Mobile Asset Controller,CP05MOAS-GL388-00-ENG000000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]"
6,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL387,Coastal Glider 387,00-ENG000000,Mobile Asset Controller,CP05MOAS-GL387-00-ENG000000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8]"
7,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL380,Coastal Glider 380,00-ENG000000,Mobile Asset Controller,CP05MOAS-GL380-00-ENG000000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]"
8,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL379,Coastal Glider 379,00-ENG000000,Mobile Asset Controller,CP05MOAS-GL379-00-ENG000000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
9,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL376,Coastal Glider 376,00-ENG000000,Mobile Asset Controller,CP05MOAS-GL376-00-ENG000000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


##### Select an ADCP and associated Engineering dataset

In [4]:
refdes = 'CP05MOAS-GL388-01-ADCPAM000'
eng_refdes = 'CP05MOAS-GL388-00-ENG000000'

### Download ADCP Data

##### Check available datastreams

In [5]:
M2M.get_datastreams(refdes)

,refdes,method,stream
0,CP05MOAS-GL388-01-ADCPAM000,recovered_host,adcp_bottom_track_config
1,CP05MOAS-GL388-01-ADCPAM000,recovered_host,adcp_bottom_track_earth
2,CP05MOAS-GL388-01-ADCPAM000,recovered_host,adcp_config
3,CP05MOAS-GL388-01-ADCPAM000,recovered_host,adcp_engineering
4,CP05MOAS-GL388-01-ADCPAM000,recovered_host,adcp_velocity_earth


In [9]:
site, node, sensor = refdes.split("-", 2)
method = 'recovered_host'

# Load the velocity data
stream = 'adcp_velocity_earth'
adcp_data = load_kdata(site, node, sensor, method, stream, tag=f'deployment0008*{refdes}*.nc')

# Load the engineering data
stream = 'adcp_engineering'
adcp_engineering = load_kdata(site, node, sensor, method, stream, tag=f'deployment0008*{refdes}*.nc')

# Load the config data
stream = 'adcp_config'
adcp_config = load_kdata(site, node, sensor, method, stream, tag=f'deployment0008*{refdes}*.nc')

# Load the bottom track data
stream = 'adcp_bottom_track_earth'
adcp_bottom_track = load_kdata(site, node, sensor, method, stream, tag=f'deployment0008*{refdes}*.nc')

Loading and Processing Data Files: 100%|██████████| 2/2 [00:12<00:00,  6.04s/it]


Merging the data files into a single dataset


Loading and Processing Data Files: 100%|██████████| 1/1 [00:00<00:00,  1.90it/s]


Merging the data files into a single dataset


Loading and Processing Data Files: 100%|██████████| 1/1 [00:00<00:00, 13.17it/s]


Merging the data files into a single dataset


Loading and Processing Data Files: 100%|██████████| 1/1 [00:00<00:00,  1.16it/s]


Merging the data files into a single dataset


###### Save the ADCP Data & associated datasets

In [18]:
# ADCP Data
adcp_data.to_netcdf(f"../data/raw/{refdes}.deployment0008.adcp_data.nc", format="netcdf4", engine="h5netcdf")

# ADCP Engineering
adcp_engineering.to_netcdf(f"../data/raw/{refdes}.deployment0008.adcp_engineering.nc", format="netcdf4", engine="h5netcdf")

# ADCP Config
adcp_config.to_netcdf(f"../data/raw/{refdes}.deployment0008.adcp_config.nc", format="netcdf4", engine="h5netcdf")

##### Glider Data
Next, need to get the glider engineering data which has the key data we will need for calculating differences between waypoints and surfacings. This is not available via netCDF file or on kdata, so will need to make a THREDDS request and download the separate csv files

In [10]:
M2M.get_datastreams(eng_refdes)

,refdes,method,stream
0,CP05MOAS-GL388-00-ENG000000,recovered_host,glider_eng_metadata_recovered
1,CP05MOAS-GL388-00-ENG000000,recovered_host,glider_eng_recovered
2,CP05MOAS-GL388-00-ENG000000,recovered_host,glider_eng_sci_recovered
3,CP05MOAS-GL388-00-ENG000000,recovered_host,glider_gps_position
4,CP05MOAS-GL388-00-ENG000000,telemetered,glider_eng_metadata
5,CP05MOAS-GL388-00-ENG000000,telemetered,glider_eng_sci_telemetered
6,CP05MOAS-GL388-00-ENG000000,telemetered,glider_eng_telemetered
7,CP05MOAS-GL388-00-ENG000000,telemetered,glider_gps_position


In [12]:
M2M.get_deployments(refdes, deploy_num="8")

,deploymentNumber,uid,assetId,latitude,longitude,depth,deployStart,deployEnd,deployCruise,recoverCruise
0,8,CGINS-ADCPAM-49016,3498,39.92227,-70.91603,1000.0,2019-06-18 22:58:15,2019-09-21 12:56:30,WJ20190617,AR38


In [13]:
from ooinet.M2M import SESSION
from bs4 import BeautifulSoup

# Request the dataset
# thredds_url = M2M.get_thredds_url(refdes, 'recovered_host', 'glider_eng_recovered', goldCopy=False, beginDT="2019-06-18 22:58:15", endDT="2019-09-21 12:56:30")
thredds_url = 'https://downloads.oceanobservatories.org/async_results/areed@whoi.edu/20250325T203718572Z-CP05MOAS-GL388-00-ENG000000-recovered_host-glider_eng_recovered/' # THREDDS request are good for ~6 months

# Access the THREDDS catalog of the available datasets
page = SESSION.get(thredds_url).text
soup = BeautifulSoup(page, "html.parser")
pattern = re.compile('.*\\.csv$')
catalog = sorted([node.get('href') for node in soup.find_all('a', text=pattern)])
catalog

/tmp/ipykernel_2280/3914584250.py:12: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  catalog = sorted([node.get('href') for node in soup.find_all('a', text=pattern)])


['deployment0008_CP05MOAS-GL388-00-ENG000000-recovered_host-glider_eng_recovered_20190618T230135-20190619T115946.csv',
 'deployment0008_CP05MOAS-GL388-00-ENG000000-recovered_host-glider_eng_recovered_20190619T120011-20190620T115955.csv',
 'deployment0008_CP05MOAS-GL388-00-ENG000000-recovered_host-glider_eng_recovered_20190620T120000-20190621T115959.csv',
 'deployment0008_CP05MOAS-GL388-00-ENG000000-recovered_host-glider_eng_recovered_20190621T120003-20190622T115957.csv',
 'deployment0008_CP05MOAS-GL388-00-ENG000000-recovered_host-glider_eng_recovered_20190622T120002-20190623T115959.csv',
 'deployment0008_CP05MOAS-GL388-00-ENG000000-recovered_host-glider_eng_recovered_20190623T120026-20190624T115956.csv',
 'deployment0008_CP05MOAS-GL388-00-ENG000000-recovered_host-glider_eng_recovered_20190624T120000-20190625T115955.csv',
 'deployment0008_CP05MOAS-GL388-00-ENG000000-recovered_host-glider_eng_recovered_20190625T120019-20190626T115958.csv',
 'deployment0008_CP05MOAS-GL388-00-ENG000000-rec

In [14]:
# load the glider engineering data
glider_engineering = pd.DataFrame()
for file in catalog:
    df = pd.read_csv(thredds_url + file)
    glider_engineering = pd.concat([glider_engineering, df], ignore_index=True)

glider_engineering.head()

,obs,c_air_pump,c_ballast_pumped,c_battpos,c_battroll,c_bsipar_on,c_de_oil_vol,c_dvl_on,c_flbbcd_on,c_heading,...,m_water_vy,m_why_started,m_x_lmc,m_y_lmc,port_timestamp,preferred_timestamp,time,x_last_wpt_lat,x_last_wpt_lon,x_system_clock_adjusted
0,"('0',)",1,260.0,0.738,0.0,-1.0,260.0,-1.0,-1.0,0.0,...,0.0,64,0.0,0.0,0.0,internal_timestamp,3.769888e+09,0.0,0.0,0.0
1,"('1',)",1,260.0,0.738,0.0,-1.0,260.0,-1.0,-1.0,0.0,...,0.0,64,0.0,0.0,0.0,internal_timestamp,3.769888e+09,0.0,0.0,0.0
2,"('2',)",1,NaN,0.738,0.0,-1.0,260.0,-1.0,-1.0,NaN,...,NaN,-99,NaN,NaN,0.0,internal_timestamp,3.769888e+09,NaN,NaN,NaN
3,"('3',)",1,NaN,0.738,0.0,-1.0,260.0,-1.0,-1.0,NaN,...,NaN,-99,NaN,NaN,0.0,internal_timestamp,3.769888e+09,NaN,NaN,NaN
4,"('4',)",1,NaN,0.738,0.0,-1.0,260.0,-1.0,-1.0,NaN,...,NaN,-99,NaN,NaN,0.0,internal_timestamp,3.769888e+09,NaN,NaN,NaN


In [16]:
from ooinet.utils import ntp_seconds_to_datetime

glider_engineering['time'] = glider_engineering['time'].apply(lambda x: ntp_seconds_to_datetime(x))
glider_engineering = glider_engineering.set_index(keys='time')
glider_engineering.head()

,obs,c_air_pump,c_ballast_pumped,c_battpos,c_battroll,c_bsipar_on,c_de_oil_vol,c_dvl_on,c_flbbcd_on,c_heading,...,m_water_vx,m_water_vy,m_why_started,m_x_lmc,m_y_lmc,port_timestamp,preferred_timestamp,x_last_wpt_lat,x_last_wpt_lon,x_system_clock_adjusted
time,,,,,,,,,,,,,,,,,,,,,
2019-06-18 23:01:35.200470,"('0',)",1,260.0,0.738,0.0,-1.0,260.0,-1.0,-1.0,0.0,...,0.0,0.0,64,0.0,0.0,0.0,internal_timestamp,0.0,0.0,0.0
2019-06-18 23:02:29.318180,"('1',)",1,260.0,0.738,0.0,-1.0,260.0,-1.0,-1.0,0.0,...,0.0,0.0,64,0.0,0.0,0.0,internal_timestamp,0.0,0.0,0.0
2019-06-18 23:03:01.304530,"('2',)",1,NaN,0.738,0.0,-1.0,260.0,-1.0,-1.0,NaN,...,NaN,NaN,-99,NaN,NaN,0.0,internal_timestamp,NaN,NaN,NaN
2019-06-18 23:03:05.753880,"('3',)",1,NaN,0.738,0.0,-1.0,260.0,-1.0,-1.0,NaN,...,NaN,NaN,-99,NaN,NaN,0.0,internal_timestamp,NaN,NaN,NaN
2019-06-18 23:03:10.206570,"('4',)",1,NaN,0.738,0.0,-1.0,260.0,-1.0,-1.0,NaN,...,NaN,NaN,-99,NaN,NaN,0.0,internal_timestamp,NaN,NaN,NaN


###### Save the glider engineering dataset

In [17]:
# Save the dataset
glider_engineering.to_csv("../data/raw/deployment0008_CP05MOAS-GL388-00-ENG000000-recovered_host-glider_eng_recovered.csv")

---
## Science Data
Next, download the associated glider data from the IOOS GDAC

In [19]:
from erddapy import ERDDAP

In [26]:
# Connect to the IOOS GDAC ERDDAP server
e = ERDDAP(server='https://gliders.ioos.us/erddap')

# Grab the search url and look for OOI gliders
url = e.get_search_url(search_for="ooi", response="csv")

# Get available OOI dataset
df = pd.read_csv(url)

# List the datasets with the glider that we want
glider_id = 'cp_388'
[x for x in df['Dataset ID'] if glider_id in x]

['cp_388-20241120T1604',
 'cp_388-20141006T2014-delayed',
 'cp_388-20151022T1232-delayed',
 'cp_388-20160809T1406-delayed',
 'cp_388-20170116T1324-delayed',
 'cp_388-20171101T0120-delayed',
 'cp_388-20181030T2211-delayed',
 'cp_388-20190618T2258-delayed',
 'cp_388-20191212T2247-delayed',
 'cp_388-20200613T2149-delayed',
 'cp_388-20210517T2025-delayed',
 'cp_388-20220104T2057-delayed',
 'cp_388-20220410T1600-delayed',
 'cp_388-20220411T1624-delayed',
 'cp_388-20200613T2149',
 'cp_388-20210517T2025',
 'cp_388-20220104T2057',
 'cp_388-20220411T1624']

In [27]:
# Set the dataset constraints
e.dataset_id = 'cp_388-20190618T2258-delayed'
e.protocol = 'tabledap'
e.constraints = None
data_url = e.get_download_url(response='opendap')

In [28]:
# Download the data
gdac = e.to_xarray()
gdac['time'] = gdac['precise_time']
gdac = gdac.drop_vars(names='precise_time')
gdac = gdac.swap_dims({'obs':'time'})
gdac.load()

<xarray.Dataset> Size: 2GB
Dimensions:                                  (trajectory: 1, profile: 1154,
                                              time: 6015211)
Coordinates:
  * trajectory                               (trajectory) object 8B 'cp_388-2...
  * time                                     (time) datetime64[ns] 48MB 2019-...
    latitude                                 (profile) float64 9kB 39.92 ... ...
    longitude                                (profile) float64 9kB -70.91 ......
    lat_uv                                   (profile) float64 9kB 39.92 ... ...
    lon_uv                                   (profile) float64 9kB -70.91 ......
    time_uv                                  (profile) datetime64[ns] 9kB 201...
    depth                                    (time) float32 24MB nan nan ... 5.2
Dimensions without coordinates: profile
Data variables: (12/56)
    wmo_id                                   (trajectory) object 8B '4801956'
    profile_id                               (profile) float64 9kB 1.0 ... 1....
    u                                        (profile) float64 9kB 0.4301 ......
    v                                        (profile) float64 9kB 0.05074 .....
    trajectoryIndex                          (profile) int32 5kB 0 0 0 ... 0 0 0
    rowSize                                  (profile) int32 5kB 568 ... 2137
    ...                                       ...
    qartod_temperature_spike_flag            (time) float32 24MB nan nan ... 1.0
    radiation_wavelength                     (time) float64 48MB 700.0 ... 700.0
    roll                                     (time) float64 48MB -1.0 ... 1.1
    salinity                                 (time) float32 24MB nan ... 35.27
    source_file                              (time) object 48MB 'cp_388-2019-...
    temperature                              (time) float32 24MB nan ... 22.97
Attributes: (12/73)
    acknowledgment:                 Funding provided by the National Science ...
    cdm_data_type:                  TrajectoryProfile
    cdm_profile_variables:          time_uv,lat_uv,lon_uv,u,v,profile_id,time...
    cdm_trajectory_variables:       trajectory,wmo_id
    comment:                        Deployment summary: This glider was deplo...
    contributor_name:               Al Plueddemann, Derek Buffitt, Peter Bric...
    ...                             ...
    time_coverage_end:              2019-09-10T23:14:23Z
    time_coverage_resolution:       PT01.075783S
    time_coverage_start:            2019-06-18T23:08:56Z
    title:                          cp_388-20190618T2258-delayed
    uuid:                           4a5228e5-d391-4137-ac19-3c33883ca9ca
    Westernmost_Easting:            -71.5674764824985

In [29]:
# Save the GDAC dataset
gdac.to_netcdf("../data/raw/CP05MOAS-GL388.deployment0008.science.nc", format='netcdf4', engine='h5netcdf')